## QUESTION 1
What's the version.build_hash value?

In [1]:
!curl http://elasticsearch:9200

{
  "name" : "0e0cdb6df403",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "5llfMMuDRQ6a4iZSALtgMw",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## A1

`"build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7"`

## Getting the Data

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

``` 
pip install elasticsearch
``` 

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

## A2
`index`

## Q3. Searching

Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 84.50
* 64.50
* 44.50
* 24.50
Look at the `_score` field.

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es_client = Elasticsearch('http://elasticsearch:9200')

In [4]:
es_client.info()

ObjectApiResponse({'name': '0e0cdb6df403', 'cluster_name': 'docker-cluster', 'cluster_uuid': '5llfMMuDRQ6a4iZSALtgMw', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'hw-course-questions'

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'hw-course-questions'})

In [7]:
from tqdm.auto import tqdm

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
for doc in tqdm(documents): 
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:05<00:00, 176.72it/s]


In [23]:
query = "How do execute a command on a Kubernetes pod?"

In [20]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
}}}

In [21]:
response = es_client.search(index=index_name, body=search_query)

In [22]:
response['hits']

{'total': {'value': 739, 'relation': 'eq'},
 'max_score': 44.50556,
 'hits': [{'_index': 'hw-course-questions',
   '_id': 'VNr6TZcBE5PBDb_RME_d',
   '_score': 44.50556,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'hw-course-questions',
   '_id': '49r6TZcBE5PBDb_RM093',
   '_score': 35.433445,
   '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
    'section': '10. Kubernetes and TensorFlow Serving',
    'question': 'Kubernetes-dashboard',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'hw-c

## A3. Searching
* `44.50`

## Q4. Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

*  How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [27]:
query = "How do copy a file to a Docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
    }

In [28]:
response = es_client.search(index=index_name, body=search_query)

In [29]:
response['hits']

{'total': {'value': 340, 'relation': 'eq'},
 'max_score': 73.38676,
 'hits': [{'_index': 'hw-course-questions',
   '_id': 'VNr6TZcBE5PBDb_RME_d',
   '_score': 73.38676,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'hw-course-questions',
   '_id': 'c9r6TZcBE5PBDb_RMU9r',
   '_score': 66.688705,
   '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can

## A4
* How do I copy files from a different folder into docker container’s working directory?

## Q5. Building a prompt


Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)

``` 
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 946
* 1446
* 1946
* 2446

In [85]:
response = es_client.search(index=index_name, body=search_query)

result_docs = []

for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

context_template = """
Q: {question}
A: {text}
""".strip()


for doc in result_docs:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
prompt = prompt_template.format(question=query, context=context).strip()

In [86]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How do copy a file to a Docker container?

    CONTEXT: 
    section: 5. Deploying Machine Learning Models
question: How do I debug a docker container?
answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

section: 5. Deploying Machine Learning Models
question: How do I copy files from my local machine to docker container?
answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can 

In [87]:
len(prompt)

4425

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```
pip install tiktoken
```

Let's calculate the number of tokens in our query:

```
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 120
* 220
* 320
* 420
  
Note: to decode back a token into a word, you can use the decode_single_token_bytes function:

```
encoding.decode_single_token_bytes(63842)
```

In [63]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 455.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 22.0 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [65]:
import tiktoken

In [66]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [81]:
tokens = encoding.encode(prompt)

In [82]:
len(tokens)

358